## Linking without deduplication

A simple record linkage model using the `link_only` [link type](https://moj-analytical-services.github.io/splink/settings_dict_guide.html#link_type).

With `link_only`, only between-dataset record comparisons are generated. No within-dataset record comparisons are created, meaning that the model does not attempt to find within-dataset duplicates.


<a target="_blank" href="https://colab.research.google.com/github/moj-analytical-services/splink/blob/splink4_dev/docs/demos/examples/duckdb/link_only.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>


In [1]:
# Uncomment and run this cell if you're running in Google Colab.
# !pip install git+https://github.com/moj-analytical-services/splink.git@splink4_dev

In [2]:
from splink import splink_datasets

df = splink_datasets.fake_1000

# Split a simple dataset into two, separate datasets which can be linked together.
df_l = df.sample(frac=0.5)
df_r = df.drop(df_l.index)

df_l.head(2)

,unique_id,first_name,surname,dob,city,email,cluster
413,413,NaN,Owen,1973-01-16,London,opoppy.owen71@diaz.co,107
571,571,NaN,Turner,1997-04-24,Glasgow,NaN,140


In [3]:
from splink import Linker, DuckDBAPI, block_on, SettingsCreator
import splink.comparison_library as cl
import splink.comparison_template_library as ctl

settings = SettingsCreator(
    link_type="link_only",
    blocking_rules_to_generate_predictions=[
        block_on("first_name"),
        block_on("surname"),
    ],
    comparisons=[
        ctl.NameComparison(
            "first_name",
        ),
        ctl.NameComparison("surname"),
        ctl.DateComparison(
            "dob",
            input_is_string=True,
            invalid_dates_as_null=True,
            datetime_metrics=["month", "year", "year"],
            datetime_thresholds=[1, 1, 10],
        ),
        cl.ExactMatch("city").configure(term_frequency_adjustments=True),
        ctl.EmailComparison("email", include_username_fuzzy_level=False),
    ],
)

linker = Linker(
    [df_l, df_r],
    settings,
    database_api=DuckDBAPI(),
    input_table_aliases=["df_left", "df_right"],
)

In [4]:
linker.completeness_chart(cols=["first_name", "surname", "dob", "city", "email"])

alt.LayerChart(...)

In [5]:
from splink.blocking_rule_library import BlockingRuleCreator

deterministic_rules = [
    "l.first_name = r.first_name and levenshtein(r.dob, l.dob) <= 1",
    "l.surname = r.surname and levenshtein(r.dob, l.dob) <= 1",
    "l.first_name = r.first_name and levenshtein(r.surname, l.surname) <= 2",
    block_on("email"),
]

# TODO: estimate_probability_two_random_records_match should accept
# BlockingRuleCreator objects
deterministic_rules = [
    br.get_blocking_rule("duckdb") if isinstance(br, BlockingRuleCreator) else br
    for br in deterministic_rules
]

linker.estimate_probability_two_random_records_match(deterministic_rules, recall=0.7)

Probability two random records match is estimated to be  0.00346.
This means that amongst all possible pairwise record comparisons, one in 289.26 are expected to match.  With 250,000 total possible comparisons, we expect a total of around 864.29 matching pairs


In [6]:
linker.estimate_u_using_random_sampling(max_pairs=1e6, seed=1)

----- Estimating u probabilities using random sampling -----



Estimated u probabilities using random sampling



Your model is not yet fully trained. Missing estimates for:
    - first_name (no m values are trained).
    - surname (no m values are trained).
    - dob (no m values are trained).
    - city (no m values are trained).
    - email (no m values are trained).


In [7]:
session_dob = linker.estimate_parameters_using_expectation_maximisation(block_on("dob"))
session_email = linker.estimate_parameters_using_expectation_maximisation(
    block_on("email")
)
session_first_name = linker.estimate_parameters_using_expectation_maximisation(
    block_on("first_name")
)


----- Starting EM training session -----



Estimating the m probabilities of the model by blocking on:
l."dob" = r."dob"

Parameter estimates will be made for the following comparison(s):
    - first_name
    - surname
    - city
    - email

Parameter estimates cannot be made for the following comparison(s) since they are used in the blocking rules: 
    - dob


Iteration 1: Largest change in params was -0.438 in the m_probability of surname, level `Exact match on surname`


Iteration 2: Largest change in params was 0.112 in probability_two_random_records_match


Iteration 3: Largest change in params was 0.0307 in probability_two_random_records_match


...

Iteration 3: Largest change in params was 0.000987 in the m_probability of email, level `All other comparisons`


Iteration 4: Largest change in params was 0.000257 in the m_probability of email, level `All other comparisons`


Iteration 5: Largest change in params was 6.56e-05 in the m_probability of email, level `All other comparisons`



EM converged after 5 iterations



Your model is fully trained. All comparisons have at least one estimate for their m and u values


In [8]:
results = linker.predict(threshold_match_probability=0.9)

In [9]:
results.as_pandas_dataframe(limit=5)

,match_weight,match_probability,source_dataset_l,source_dataset_r,unique_id_l,unique_id_r,first_name_l,first_name_r,gamma_first_name,surname_l,...,dob_l,dob_r,gamma_dob,city_l,city_r,gamma_city,email_l,email_r,gamma_email,match_key
0,3.208019,0.902351,df_left,df_right,790,791,Jackson,Jackson,3,Fisreh,...,2004-12-26,2003-12-28,2,Lodon,Londonn,0,j.fisher4@sullivan.com,None,-1,0
1,3.229125,0.903633,df_left,df_right,541,542,Harry,Harry,3,None,...,2019-03-16,2019-04-15,3,York,None,-1,None,harryandrews@woods.bz,-1,0
2,3.229125,0.903633,df_left,df_right,525,526,George,George,3,None,...,1980-06-29,1980-06-30,3,None,Lndon,-1,None,go94@brndshaw-saaders.com,-1,0
3,3.285750,0.906997,df_left,df_right,324,325,Kai,Kai,3,None,...,2018-12-31,2008-12-03,0,London,London,1,k.t50eherand@z.ncom,k.t50@hernandez.com,1,0
4,3.449200,0.916123,df_left,df_right,426,425,None,Alfie,-1,Roberts,...,2005-02-08,1995-01-10,0,London,London,1,ar20@miller-tores.org,ar20@miller-torres.org,2,1
